In [16]:
!pip install pandas scikit-learn


In [17]:
import pandas as pd

# Load the train.csv file
train_path = './content/train.csv'
train_data = pd.read_csv(train_path)

# Display the first few rows to understand its structure
train_data.head()



,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,1092.0,NaN,14610.0,F,NaN,NaN,NaN,N,1.8,NaN,3.65,NaN,NaN,NaN,NaN,269.0,10.5,3.0,C
1,1,1783.0,Placebo,22646.0,F,N,N,N,N,0.5,328.0,3.80,14.0,1044.0,85.25,108.0,271.0,10.3,2.0,C
2,2,130.0,Placebo,13918.0,F,Y,Y,Y,Y,5.6,NaN,3.07,182.0,1029.0,119.35,NaN,399.0,11.7,4.0,D
3,3,1725.0,Placebo,11462.0,F,N,Y,N,N,0.6,376.0,3.26,45.0,794.0,82.15,154.0,293.0,10.7,4.0,C
4,4,2812.0,NaN,18628.0,F,NaN,NaN,NaN,N,0.7,NaN,3.40,NaN,NaN,NaN,NaN,166.0,9.9,1.0,C


In [18]:
# Reload the datasets
train_path = './content/train.csv'
test_path = './content/test.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

# Check for missing values in the reloaded datasets
missing_values_train = train_data.isnull().sum()
missing_values_test = test_data.isnull().sum()

# Summarize missing values
missing_summary_train = missing_values_train[missing_values_train > 0]
missing_summary_test = missing_values_test[missing_values_test > 0]

missing_summary_train, missing_summary_test


(N_Days              1
 Drug             6478
 Ascites          6466
 Hepatomegaly     6474
 Spiders          6487
 Cholesterol      8361
 Copper           6585
 Alk_Phos         6483
 SGOT             6487
 Tryglicerides    8414
 Platelets         607
 Prothrombin        20
 dtype: int64,
 Drug             4483
 Ascites          4476
 Hepatomegaly     4485
 Spiders          4486
 Cholesterol      5670
 Copper           4542
 Alk_Phos         4487
 SGOT             4488
 Tryglicerides    5698
 Platelets         400
 Prothrombin        12
 dtype: int64)

In [19]:
# Exclude 'Status' column from the categorical columns list for the test dataset
categorical_cols = [col for col in categorical_cols if col != 'Status']

# Impute numerical features with median
train_data[numerical_cols] = numerical_imputer.fit_transform(train_data[numerical_cols])
test_data[numerical_cols] = numerical_imputer.transform(test_data[numerical_cols])

# Impute categorical features with the most frequent value
train_data[categorical_cols] = categorical_imputer.fit_transform(train_data[categorical_cols])
test_data[categorical_cols] = categorical_imputer.transform(test_data[categorical_cols])

# Verify if all missing values are handled
missing_values_train_after = train_data.isnull().sum().sum()
missing_values_test_after = test_data.isnull().sum().sum()

missing_values_train_after, missing_values_test_after


NameError: name 'categorical_cols' is not defined

In [ ]:
print("Remaining missing values in training data:", train_data.isnull().sum().sum())
print("Remaining missing values in test data:", test_data.isnull().sum().sum())

Remaining missing values in training data: 0
Remaining missing values in test data: 0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, accuracy_score


In [ ]:
# Separate features and target
X = train_data.drop(columns=["Status", "id"])  # Drop the target and ID columns
y = train_data["Status"]

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Define preprocessing for categorical and numerical data
categorical_cols = X.select_dtypes(include=["object"]).columns
numerical_cols = X.select_dtypes(exclude=["object"]).columns

# One-hot encode categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numerical_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
    ]
)

# Transform the training and validation datasets
X_train_encoded = preprocessor.fit_transform(X_train)
X_val_encoded = preprocessor.transform(X_val)



In [ ]:
# Train the Random Forest model on the encoded data
rf_model.fit(X_train_encoded, y_train)



RandomForestClassifier(n_estimators=200, random_state=42)

In [ ]:
# Predict probabilities
y_val_pred_probs = rf_model.predict_proba(X_val)

# Evaluate using log-loss
val_log_loss = log_loss(y_val, y_val_pred_probs)

print("Validation Log-Loss:", val_log_loss)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'D-penicillamine'

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=["object"]).columns
numerical_cols = X.select_dtypes(exclude=["object"]).columns

# Define preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numerical_cols),  # Keep numerical columns as is
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),  # Encode categorical columns
    ]
)

# Create a pipeline with preprocessing and model
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42, n_estimators=200))
])


In [ ]:
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  Index(['N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper',
       'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin',
       'Stage'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema'], dtype='object'))])),
                ('classifier',
                 RandomForestClassifier(n_estimators=200, random_state=42))])

In [ ]:
# Predict probabilities for the validation set
y_val_pred_probs = pipeline.predict_proba(X_val)

# Evaluate using log-loss
from sklearn.metrics import log_loss
val_log_loss = log_loss(y_val, y_val_pred_probs)

print("Validation Log-Loss:", val_log_loss)


Validation Log-Loss: 0.4220142569935712


In [ ]:
param_grid = {
    "classifier__n_estimators": [100, 200, 300],  # Number of trees
    "classifier__max_depth": [10, 20, None],      # Maximum depth of the trees
    "classifier__min_samples_split": [2, 5, 10],  # Minimum samples to split an internal node
    "classifier__min_samples_leaf": [1, 2, 4],    # Minimum samples at a leaf node
}


In [ ]:
from sklearn.model_selection import GridSearchCV

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring="neg_log_loss",  # Use log-loss as the evaluation metric
    cv=3,                    # 3-fold cross-validation
    verbose=2,
    n_jobs=-1                # Use all available cores
)

# Run the grid search
grid_search.fit(X_train, y_train)

# Get the best parameters
best_pipeline = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)


Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best Parameters: {'classifier__max_depth': 20, 'classifier__min_samples_leaf': 4, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 300}


In [ ]:
# Predict probabilities for the validation set
y_val_pred_probs = best_pipeline.predict_proba(X_val)

# Evaluate using log-loss
from sklearn.metrics import log_loss
val_log_loss = log_loss(y_val, y_val_pred_probs)

print("Validation Log-Loss after Tuning:", val_log_loss)


Validation Log-Loss after Tuning: 0.37550686999084154


In [ ]:
# Predict probabilities for the test set
test_pred_probs = best_pipeline.predict_proba(test_data.drop(columns=["id"]))

# Create submission file
submission = pd.DataFrame({
    "id": test_data["id"],
    "Status_C": test_pred_probs[:, best_pipeline.classes_.tolist().index("C")],
    "Status_CL": test_pred_probs[:, best_pipeline.classes_.tolist().index("CL")],
    "Status_D": test_pred_probs[:, best_pipeline.classes_.tolist().index("D")]
})

# Save the submission file
submission.to_csv("submission1.csv", index=False)


# xgboost

In [ ]:
!pip install xgboost


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Display the mapping for reference
print("Class Mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))


Class Mapping: {'C': 0, 'CL': 1, 'D': 2}


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder

# Identify categorical and numerical columns
categorical_cols = X_train.select_dtypes(include=["object"]).columns
numerical_cols = X_train.select_dtypes(exclude=["object"]).columns

# Define preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numerical_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
    ]
)

# Transform training and validation sets
X_train_encoded = preprocessor.fit_transform(X_train)
X_val_encoded = preprocessor.transform(X_val)

# Encode target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)


In [ ]:
from xgboost import XGBClassifier

# Initialize XGBoost model
xgb_model = XGBClassifier(
    objective="multi:softprob",
    eval_metric="mlogloss",
    random_state=42
)

# Train the model
xgb_model.fit(X_train_encoded, y_train_encoded)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The XGBClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
from sklearn.metrics import log_loss

# Predict probabilities
y_val_pred_probs = xgb_model.predict_proba(X_val_encoded)

# Evaluate log-loss
val_log_loss = log_loss(y_val_encoded, y_val_pred_probs)
print("Validation Log-Loss with XGBoost:", val_log_loss)


Validation Log-Loss with XGBoost: 0.37083772287161854


In [ ]:
# Transform test data
X_test_encoded = preprocessor.transform(test_data.drop(columns=["id"]))

# Predict probabilities for the test set
test_pred_probs = xgb_model.predict_proba(X_test_encoded)

# Create submission file
submission = pd.DataFrame({
    "id": test_data["id"],
    "Status_C": test_pred_probs[:, label_encoder.transform(["C"])[0]],
    "Status_CL": test_pred_probs[:, label_encoder.transform(["CL"])[0]],
    "Status_D": test_pred_probs[:, label_encoder.transform(["D"])[0]],
})

# Save submission file
submission.to_csv("submission_xgboost.csv", index=False)
print("Submission file saved as 'submission_xgboost.csv'")


Submission file saved as 'submission_xgboost.csv'


In [ ]:
# Create submission file
submission = pd.DataFrame({
    "id": test_data["id"],
    "Status_C": test_pred_probs[:, label_encoder.transform(["C"])[0]],
    "Status_CL": test_pred_probs[:, label_encoder.transform(["CL"])[0]],
    "Status_D": test_pred_probs[:, label_encoder.transform(["D"])[0]],
})

# Save the submission file
submission_file = "submission_xgboost.csv"
submission.to_csv(submission_file, index=False)
print(f"Submission file saved as '{submission_file}'")


Submission file saved as 'submission_xgboost.csv'


In [ ]:
print(submission.head())


        id  Status_C  Status_CL  Status_D
0  15000.0  0.989089   0.000253  0.010657
1  15001.0  0.694177   0.005261  0.300562
2  15002.0  0.020366   0.026760  0.952874
3  15003.0  0.982835   0.000525  0.016640
4  15004.0  0.977500   0.000042  0.022458
